In [1]:
import pandas as pd
import numpy as np 
import pandas_datareader
from yahoofinancials import YahooFinancials
import json
import warnings
import os
warnings.filterwarnings('ignore')
import plotly.express as px
import plotly.graph_objects as go

In [2]:
df_infos = pd.read_excel("empresas.xlsx")
lista = list(df_infos['Simbolo'])

In [3]:
lista[31]

'EMBR3.SA'

In [4]:
tickers = [lista[31]]
yahoo_financials = YahooFinancials(tickers)
income = yahoo_financials.get_financial_stmts('annual', 'income', reformat=True)
income

{'incomeStatementHistory': {'EMBR3.SA': [{'2019-12-31': {'gainOnSaleOfSecurity': 6900000.0,
     'otherGandA': 190200000.0,
     'netInterestIncome': -116100000.0,
     'netNonOperatingInterestIncomeExpense': -116100000.0,
     'taxRateForCalcs': 0.34,
     'minorityInterests': -5800000.0,
     'dilutedAverageShares': 735850000.0,
     'reconciledCostOfRevenue': 4667100000.0,
     'sellingAndMarketingExpense': 285900000.0,
     'otherTaxes': 32900000.0,
     'dilutedNIAvailtoComStockholders': -322300000.0,
     'normalizedIncome': -326854000.0,
     'ebit': -76800000.0,
     'totalUnusualItems': 6900000.0,
     'netIncomeDiscontinuousOperations': -111800000.0,
     'interestExpenseNonOperating': 15500000.0,
     'costOfRevenue': 4667100000.0,
     'netIncome': -322300000.0,
     'pretaxIncome': -186200000.0,
     'operatingIncome': -76800000.0,
     'netIncomeFromContinuingAndDiscontinuedOperation': -322300000.0,
     'sellingGeneralAndAdministration': 476100000.0,
     'totalUnusualIt

In [5]:
def dropnested(lista):
    outputdict = {}
    dates = []
    for dic in lista:
        for key, value in dic.items():
            dates.append(key.replace('-',''))
            if isinstance(value, dict):
                for k2, v2, in value.items():
                    outputdict[k2] = outputdict.get(k2, []) + [v2]
            else:
                outputdict[key] = outputdict.get(key, []) + [value]
    
    return outputdict, dates 

tickers = [lista[31]]
yahoo_financials = YahooFinancials(tickers)
income = yahoo_financials.get_financial_stmts('annual', 'income', reformat=True)
data = income['incomeStatementHistory']
df =  pd.DataFrame()
for ticker in tickers:
    t_data = data[ticker]
    outputdict, dates = dropnested(t_data)
    _df = pd.DataFrame.from_dict(outputdict, orient='index').T.apply(pd.to_numeric)
    _df['Simbolo'] = ticker
    end_date = pd.Series(dates, name='end_date')
    norm_df = pd.concat([_df, end_date], axis=1)
    norm_df = norm_df.set_index('end_date')
    df = df.append(norm_df)
df = df.reset_index()
df = df.replace(np.nan, 0)
df = df.query("end_date in ['20191231','20201231','20211231','20221231','20231231','20241231','20251231','20261231']")

In [ ]:
df

In [ ]:
label = {'end_date':'Período','interestIncome':'Receita Financeira','netIncome': 'Lucro Liquido','interestExpense':'Despesas Financeiras/Juros',
'interestIncomeNonOperating':'Receita Não-Operacional','specialIncomeCharges':'Receitas/Despesas Extraordinárias',
'pretaxIncome':'Lucro Antes dos Impostos','otherOperatingExpenses':'Outras Despesas Operacionais',
'sellingGeneralAndAdministration':'Despesas Vendas, Gerais e Administrativas',
 'generalAndAdministrativeExpense':'Despesas Gerais e Administrativas','totalOtherFinanceCost':'Custos Financeiros Totais',
'normalizedEBITDA':'EBITDA','operatingExpense':'Despesas Operacionais','totalExpenses':'Despesas Totais',
'restructuringAndMergernAcquisition':'Reestruturação e Fusão/Aquisição','totalUnusualItems':'Despesas e Receitas incomuns',
'ebit':'Lucro Antes de Juros e Impostos','reconciledDepreciation':'Depreciação','netInterestIncome':'Receita líquida de juros',
'otherSpecialCharges':'Outras Despesas Especiais','costOfRevenue':'Custo Produção/Serviços',
 'operatingRevenue':'Receita Operacional','grossProfit':'Lucro Bruto','totalRevenue':'Receita Bruta',
 'interestExpenseNonOperating':'Despesa Financeira','operatingIncome':'Lucro Operacional',
 'sellingAndMarketingExpense':'Despesas de Vendas e Marketing','earningsFromEquityInterest':'Resultado Equity Interest',
'netIncomeContinuousOperations':'Lucro Líquido Oper_Continuadas','otherGandA':'Outras Despesas Gerais e Adm'}

In [ ]:
df.rename(columns=label,inplace=True)

In [ ]:
'Custo Produção/Serviços' in df.columns

In [ ]:
df.columns

In [ ]:
df[['Despesas Financeiras/Juros','Despesas Gerais e Administrativas','Despesas Vendas, Gerais e Administrativas']]

In [ ]:
df['Período'] = pd.to_datetime(df['Período'], format='%Y%m%d')
df.sort_values(by='Período',inplace=True)

In [ ]:
df = df[['Período','Lucro Liquido',
    'Despesas Financeiras/Juros', 'Receita Não-Operacional',
    'Lucro Antes dos Impostos','Custos Financeiros Totais','Outras Despesas Operacionais',
    'Despesas Gerais e Administrativas','Despesas Vendas, Gerais e Administrativas',
    'EBITDA', 'Despesas Operacionais', 'Despesas Totais','Outras Despesas Especiais',
    'Lucro Antes de Juros e Impostos', 'Depreciação',
    'Receita líquida de juros', 'Receita Operacional', 'Lucro Bruto', 'Custo Produção/Serviços',
    'Receita Bruta', 'Despesa Financeira', 'Lucro Operacional',
    'Despesas de Vendas e Marketing', 'Resultado Equity Interest',
    'Outras Despesas Gerais e Adm', 'Lucro Líquido Oper_Continuadas',
    'Simbolo']]

In [ ]:
df_receita_bruta = df[['Período','Receita Operacional','Receita Não-Operacional']]
df_receita_bruta["Pecentual_Crescimento"] = round(100*df['Receita Bruta'].pct_change(),2)
df_receita_bruta = df_receita_bruta.replace(np.nan, 0)
df_receita_bruta['Crescimento'] = ['Positivo' if x > 0 else 'Negativo' for x in df_receita_bruta['Pecentual_Crescimento']]

In [ ]:
df_receita_bruta

In [ ]:
import numpy as np
fig=px.line(
            df_receita_bruta.sort_values(by='Período'),
            x="Período",
            y="Receita Operacional",
            color_discrete_sequence=['white'],
            text=df_receita_bruta["Receita Operacional"].apply(lambda x: np.format_float_positional(x/1e9, precision=4, unique=False, fractional=False, trim='k') + 'bi'),
            title="Faturamento Bruto"
).update_yaxes(
            tickprefix="R$ ",
            title="",
            showgrid=False,
).update_xaxes(
            title="Período",
            showgrid=False
).update_layout(
            #plot_bgcolor="black",
            #paper_bgcolor="black",
            #font_color="lightgrey"
            template="plotly_dark"
)
fig.update_traces(textposition="top center",textfont=dict(color='white'))
fig.show()

In [ ]:
fig = px.bar(df_receita_bruta, x='Período', y='Pecentual_Crescimento', color='Crescimento',
                color_discrete_map={"Positivo": "green", "Negativo": "red"},
                title=f"% Crescimento/Redução Receita Bruta", text_auto='% .3s'
).update_yaxes(
            title='%',
            tickprefix="% ",
).update_layout(
            #plot_bgcolor="black",
            #paper_bgcolor="white",
            #font_color="lightgrey"
            template="plotly_dark"
)
fig.update_layout(coloraxis_colorbar_title=None, xaxis_tickformat='%Y')

fig.show()

In [ ]:
import plotly.express as px

fig = px.bar(df_receita_bruta, x='Período', y=['Receita Operacional','Receita Não-Operacional'],
                color_discrete_map={"Receita Operacional": "green", "Receita Não-Operacional": "yellow"},
                title=f"Evolução Receita Operacional e Receita Não Operacional"
).update_yaxes(
            title='',
            tickprefix="R$ ",
).update_layout(
            #plot_bgcolor="black",
            #paper_bgcolor="white",
            #font_color="lightgrey"
            template="plotly_dark"
)
fig.update_layout(coloraxis_colorbar_title=None, xaxis_tickformat='%Y')

fig.show()

#### CUSTOS ####

In [ ]:
df_custos = df[['Período','Custo Produção/Serviços','Receita Operacional']]
df_custos["Pecentual_Crescimento"] = round(100*df_custos['Custo Produção/Serviços'].pct_change(),2)
df_custos = df_custos.replace(np.nan, 0)
df_custos['Crescimento'] = ['Redução' if x < 0 else 'Aumento' for x in df_custos['Pecentual_Crescimento']]
df_custos['%Custo_Receita'] = round(100*df_custos['Custo Produção/Serviços']/df_custos['Receita Operacional'],2)

In [ ]:
import numpy as np
fig=px.line(
            df_custos.sort_values(by='Período'),
            x="Período",
            y='Custo Produção/Serviços',
            color_discrete_sequence=['white'],
            text=df_custos["Custo Produção/Serviços"].apply(lambda x: np.format_float_positional(x/1e9, precision=4, unique=False, fractional=False, trim='k') + 'bi'),
            title="Evolução do Custo Produção/Serviços"
).update_yaxes(
            tickprefix="R$ ",
            title="",
            showgrid=False,
).update_xaxes(
            title="Período",
            showgrid=False
).update_layout(
            #plot_bgcolor="black",
            #paper_bgcolor="black",
            #font_color="lightgrey"
            template="plotly_dark"
)
fig.update_traces(textposition="top center",textfont=dict(color='white'))
fig.show()

In [ ]:
import plotly.express as px

fig = px.bar(df_custos, x='Período', y='Pecentual_Crescimento', color='Crescimento',
                color_discrete_map={"Redução": "green", "Aumento": "red"},
                title=f"% Aumento/Redução Custo Produção/Serviços", text_auto='% .3s'
).update_yaxes(
            title='%',
            tickprefix="% ",
).update_layout(
            #plot_bgcolor="black",
            #paper_bgcolor="white",
            #font_color="lightgrey"
            template="plotly_dark"
)
fig.update_layout(coloraxis_colorbar_title=None, xaxis_tickformat='%Y')

fig.show()

In [ ]:
import plotly.express as px

fig = px.bar(df_custos, x='Período', y='%Custo_Receita',
                color_discrete_sequence=['red'],
                title=f"% do Custo sobre a Receita Operacional", text_auto=True
).update_yaxes(
            title='',
            tickprefix="% ",
).update_layout(
            #plot_bgcolor="black",
            #paper_bgcolor="white",
            #font_color="lightgrey"
            template="plotly_dark"
)
fig.update_layout(coloraxis_colorbar_title=None, xaxis_tickformat='%Y')

fig.show()

# Despesas

In [ ]:
df_despesas = df[['Período','Receita Operacional','Despesas Totais',
                  'Despesas Gerais e Administrativas','Despesas de Vendas e Marketing',
                  'Despesas Operacionais','Despesa Financeira',
                  'Outras Despesas Gerais e Adm','Outras Despesas Especiais']]
df_despesas['Outras Despesas'] = df_despesas['Despesas Totais'] - (df_despesas['Despesas Gerais e Administrativas']+
                                                                   df_despesas['Despesas de Vendas e Marketing']+
                                                                   df_despesas['Despesas Operacionais']+
                                                                   df_despesas['Despesa Financeira']+
                                                                   df_despesas['Outras Despesas Gerais e Adm']+
                                                                   df_despesas['Outras Despesas Especiais'])
df_despesas['Despesas Gerais e Adm'] = df_despesas['Despesas Gerais e Administrativas']+ df_despesas['Outras Despesas Gerais e Adm']
df_despesas.drop(columns=['Despesas Gerais e Administrativas','Outras Despesas Gerais e Adm'], inplace=True)
df_despesas["Pecentual_Crescimento"] = round(100*df_despesas['Despesas Totais'].pct_change(),2)
df_despesas = df_despesas.replace(np.nan, 0)
df_despesas['Crescimento'] = ['Redução' if x < 0 else 'Aumento' for x in df_despesas['Pecentual_Crescimento']]
df_despesas['%Despesa_Receita'] = round(100*df_despesas['Despesas Totais']/df_despesas['Receita Operacional'],2)

In [ ]:
df_despesas

In [ ]:
import numpy as np
fig=px.line(
            df_despesas.sort_values(by='Período'),
            x="Período",
            y='Despesas Totais',
            color_discrete_sequence=['white'],
            text=df_despesas["Despesas Totais"].apply(lambda x: np.format_float_positional(x/1e9, precision=4, unique=False, fractional=False, trim='k') + 'bi'),
            title="Evolução das Despesas"
).update_yaxes(
            tickprefix="R$ ",
            title="",
            showgrid=False,
).update_xaxes(
            title="Período",
            showgrid=False
).update_layout(
            #plot_bgcolor="black",
            #paper_bgcolor="black",
            #font_color="lightgrey"
            template="plotly_dark"
)
fig.update_traces(textposition="top center",textfont=dict(color='white'))
fig.show()

In [ ]:
import plotly.express as px

fig = px.bar(df_despesas, x='Período', y='Pecentual_Crescimento', color='Crescimento',
                color_discrete_map={"Redução": "green", "Aumento": "red"},
                title=f"% Aumento/Redução Despesas", text_auto='% .3s'
).update_yaxes(
            title='%',
            tickprefix="% ",
).update_layout(
            #plot_bgcolor="black",
            #paper_bgcolor="white",
            #font_color="lightgrey"
            template="plotly_dark"
)
fig.update_layout(coloraxis_colorbar_title=None, xaxis_tickformat='%Y')

fig.show()

In [ ]:
df_despesas.columns

In [ ]:
trace1 = go.Bar(
    x=df_despesas['Período'],
    y=df_despesas['Despesas Gerais e Adm'],
    name = 'Gerais e Adm',
    marker=dict(color='#FFD700')
)
trace2 = go.Bar(
    x=df_despesas['Período'],
    y=df_despesas['Despesas de Vendas e Marketing'],
    name='Vendas e Marketing',
    marker=dict(color='#9EA0A1')
)
trace3 = go.Bar(
    x=df_despesas['Período'],
    y=df_despesas['Despesas Operacionais'],
    name='Operacionais',
    marker=dict(color='#CD7F32')
)
trace4 = go.Bar(
    x=df_despesas['Período'],
    y=df_despesas['Outras Despesas'],
    name='Outras',
    marker=dict(color='#AD7F32')
)
trace5 = go.Bar(
    x=df_despesas['Período'],
    y=df_despesas['Despesa Financeira'],
    name='Financeiras',
    marker=dict(color='#CB7F32')
)
trace6 = go.Bar(
    x=df_despesas['Período'],
    y=df_despesas['Outras Despesas Especiais'],
    name='Despesas Especiais',
    marker=dict(color='#AC2F32')
)
data = [trace1, trace2, trace3, trace4, trace5,trace6]
layout = go.Layout(
    title='Analise do Tipo da Despesa',
    barmode='stack'
)
fig = go.Figure(data=data, layout=layout)
fig.show()

In [ ]:
df.columns

In [ ]:
df_lucro = df[['Período','EBITDA','Receita Operacional']]
df_lucro["Pecentual_Crescimento"] = round(100*df_lucro['EBITDA'].pct_change(),2)
df_lucro = df_lucro.replace(np.nan, 0)
df_lucro['Crescimento'] = ['Positivo' if x > 0 else 'Negativo' for x in df_lucro['Pecentual_Crescimento']]
df_lucro['%EBITDA_Receita'] = round(100*df_lucro['EBITDA'] / df_lucro['Receita Operacional'],2)

In [ ]:
df_lucro

In [ ]:
fig=px.line(
            df_lucro.sort_values(by='Período'),
            x="Período",
            y='EBITDA',
            color_discrete_sequence=['white'],
            text=df_lucro['EBITDA'].apply(lambda x: np.format_float_positional(x/1e9, precision=4, unique=False, fractional=False, trim='k') + 'bi'),
            title="Evolução do EBITDA"
).update_yaxes(
            tickprefix="R$ ",
            title="",
            showgrid=False,
).update_xaxes(
            title="Período",
            showgrid=False
).update_layout(
            #plot_bgcolor="black",
            #paper_bgcolor="black",
            #font_color="lightgrey"
            template="plotly_dark"
)
fig.update_traces(textposition="top center",textfont=dict(color='white'))
fig.show()

In [ ]:
fig = px.bar(df_lucro, x='Período', y='Pecentual_Crescimento', color='Crescimento',
                color_discrete_map={"Positivo": "green", "Negativo": "red"},
                title=f"% de Crescimento/Redução EBITDA", text_auto='% .3s'
).update_yaxes(
            title='%',
            tickprefix="% ",
).update_layout(
            #plot_bgcolor="black",
            #paper_bgcolor="white",
            #font_color="lightgrey"
            template="plotly_dark"
)
fig.update_layout(coloraxis_colorbar_title=None, xaxis_tickformat='%Y')

fig.show()

In [ ]:
fig = px.bar(df_lucro, x='Período', y='%EBITDA_Receita',
                color_discrete_sequence=['green'],
                title=f"% do EBITDA sobre a Receita Operacional", text_auto=True
).update_yaxes(
            title='',
            tickprefix="% ",
).update_layout(
            #plot_bgcolor="black",
            #paper_bgcolor="white",
            #font_color="lightgrey"
            template="plotly_dark"
)
fig.update_layout(coloraxis_colorbar_title=None, xaxis_tickformat='%Y')

fig.show()

In [ ]:
df_lucro_liq = df[['Período','Lucro Liquido','Receita Operacional']]
df_lucro_liq["Pecentual_Crescimento"] = round(100*df_lucro_liq['Lucro Liquido'].pct_change(),2)
df_lucro_liq = df_lucro_liq.replace(np.nan, 0)
df_lucro_liq['Crescimento'] = ['Positivo' if x > 0 else 'Negativo' for x in df_lucro_liq['Pecentual_Crescimento']]
df_lucro_liq['%LucroLiquido_Receita'] = round(100*df_lucro_liq['Lucro Liquido'] / df_lucro_liq['Receita Operacional'],2)

In [ ]:
fig=px.line(
            df_lucro_liq.sort_values(by='Período'),
            x="Período",
            y='Lucro Liquido',
            color_discrete_sequence=['white'],
            text=df_lucro_liq['Lucro Liquido'].apply(lambda x: np.format_float_positional(x/1e9, precision=4, unique=False, fractional=False, trim='k') + 'bi'),
            title="Evolução do Lucro Liquido"
).update_yaxes(
            tickprefix="R$ ",
            title="",
            showgrid=False,
).update_xaxes(
            title="Período",
            showgrid=False
).update_layout(
            #plot_bgcolor="black",
            #paper_bgcolor="black",
            #font_color="lightgrey"
            template="plotly_dark"
)
fig.update_traces(textposition="top center",textfont=dict(color='white'))
fig.show()

In [ ]:
fig = px.bar(df_lucro_liq, x='Período', y='Pecentual_Crescimento', color='Crescimento',
                color_discrete_map={"Positivo": "green", "Negativo": "red"},
                title=f"% de Crescimento/Redução Lucro Liquido", text_auto='% .3s'
).update_yaxes(
            title='%',
            tickprefix="% ",
).update_layout(
            #plot_bgcolor="black",
            #paper_bgcolor="white",
            #font_color="lightgrey"
            template="plotly_dark"
)
fig.update_layout(coloraxis_colorbar_title=None, xaxis_tickformat='%Y')

fig.show()

In [ ]:
fig = px.bar(df_lucro_liq, x='Período', y='%LucroLiquido_Receita',
                color_discrete_sequence=['green'],
                title=f"% do Lucro Liquido sobre a Receita Operacional", text_auto=True
).update_yaxes(
            title='',
            tickprefix="% ",
).update_layout(
            #plot_bgcolor="black",
            #paper_bgcolor="white",
            #font_color="lightgrey"
            template="plotly_dark"
)
fig.update_layout(coloraxis_colorbar_title=None, xaxis_tickformat='%Y')

fig.show()